In [ ]:
import pandas as pd

housing_inventory = pd.read_csv('./data/HOSINVUSM495N.csv')
housing_affordability_index = pd.read_csv('./data/FIXHAI.csv')
housing_active_listings = pd.read_csv('./data/ACTLISCOUUS.csv')
housing_price_index = pd.read_csv('./data/USSTHPI.csv')
housing_vacant_units = pd.read_csv('./data/EVACANTUSQ176N.csv')
housing_median_days_on_market = pd.read_csv('./data/MEDDAYONMARUS.csv')
housing_30_year_fixed = pd.read_csv('./data/OBMMIFHA30YF.csv')
housing_median_price = pd.read_csv('./data/MSPUS.csv')

def normalize_col(df, col_name):
    mean = df.mean().loc[col_name]  
    std = df.std().loc[col_name]
    df[col_name] = df[col_name].apply(lambda x: (x - mean) / std)

    return df

def get_combined_df(df_list, months):
    combined_df = None
    
    for df in df_list:
        start = df['DATE'].size - months
        df = df[start:]
        df = normalize_col(df, df.columns[1])
        print(df.head())
        
        combined_df = df if combined_df is None else combined_df.merge(df, on='DATE', how='outer')
        
    return combined_df

dataframes = [housing_inventory, housing_affordability_index, housing_active_listings, housing_price_index, housing_vacant_units, housing_median_days_on_market, housing_30_year_fixed, housing_median_price]

df_combined = get_combined_df(dataframes, 60)
df_combined = df_combined.rename(columns={'HOSINVUSM495N':'housing_inventory', 'FIXHAI':'housing_affordability_index', 'ACTLISCOUUS':'housing_active_listings', 'USSTHPI':'housing_price_index', 'EVACANTUSQ176N':'housing_vacant_units', 'MEDDAYONMARUS':'housing_median_days_on_market', 'OBMMIFHA30YF':'housing_30_year_fixed', 'MSPUS':'housing_median_price'})
df_combined = df_combined.melt(id_vars=['DATE'],var_name='INDEX')